In [1]:
import sys
sys.path.insert(0, '..')
sys.path.insert(0, '/Users/robinlinacre/Documents/data_linking/splink/')

## Linking banking transactions

This example shows how to perform a one-to-one link on banking transactions.  The data has the following features:

- Money shows up in the destination account with some time delay
- The amount sent - which we can imagine is sometimes denominated in a foreign currency - and the amount received are not always the same
- The memo is sometimes truncated and content is sometimes missing



In [2]:
from pyarrow import parquet as pq
df_origin = pq.read_table("../data/transactions_left.parquet")
df_destination = pq.read_table("../data/transactions_right.parquet")
display(df_origin.to_pandas().head(3))
display(df_destination.to_pandas().head(3))

,ground_truth,memo,transaction_date,amount,unique_id
0,0,MATTHIAS C paym,2022-03-28,36.36,0
1,1,M CORVINUS dona,2022-02-14,221.91,1
2,2,M C donation BG,2022-05-04,449.79,2


,ground_truth,memo,transaction_date,amount,unique_id
0,0,MATTHIAS C payment BGC,2022-03-29,36.36,0
1,1,M CORVINUS BGC,2022-02-16,221.91,1
2,2,M C,2022-05-05,449.79,2


In [3]:
from splink.duckdb.duckdb_linker import DuckDBLinker

blocking_rule_date_1 = """
    strftime(l.transaction_date, '%Y%m') = strftime(r.transaction_date, '%Y%m')
    and substr(l.memo, 1,3) = substr(r.memo,1,3)
    and l.amount/r.amount > 0.9   and l.amount/r.amount < 1.1
"""

blocking_rule_date_2 = """
    strftime(l.transaction_date+15, '%Y%m') = strftime(r.transaction_date, '%Y%m')
    and substr(l.memo, 1,3) = substr(r.memo,1,3)
    and l.amount/r.amount > 0.9   and l.amount/r.amount < 1.1
"""

from splink.duckdb import duckdb_comparison_library as cl
settings = {
    "link_type": "link_only",
    "probability_two_random_records_match" : 1/len(df_origin),
    "blocking_rules_to_generate_predictions": [
        blocking_rule_date_1,
        blocking_rule_date_2
    ],
    "comparisons": [
        cl.exact_match("amount", term_frequency_adjustments=False),
        cl.jaccard_at_thresholds("memo",[0.95, 0.9, 0.7],  term_frequency_adjustments=True),
        cl.exact_match("transaction_date",  term_frequency_adjustments=True),
    ],       
    
}

In [4]:
linker = DuckDBLinker([df_origin, df_destination], settings,input_table_aliases=["org", "dst"])

In [5]:
linker.count_num_comparisons_from_blocking_rule(blocking_rule_date_1)

125423

In [6]:
df_deterministic = linker.deterministic_link()
df_deterministic.as_pandas_dataframe(limit=5)

,source_dataset_l,unique_id_l,source_dataset_r,unique_id_r,amount_l,amount_r,memo_l,memo_r,tf_memo_l,tf_memo_r,transaction_date_l,transaction_date_r,tf_transaction_date_l,tf_transaction_date_r,match_key
0,dst,1,org,44509,221.91,207.47,M CORVINUS BGC,M C BGC,0.000011,0.000077,2022-02-16,2022-02-04,0.005527,0.003751,0
1,dst,46,org,45165,73.24,67.97,A D T E payment,A D payment,0.000022,0.000066,2022-05-06,2022-05-03,0.014296,0.013921,0
2,dst,69,org,69,142.87,140.11,FILIPPINO LIPPI BGC,FILIPPINO LIPPI,0.000011,0.000055,2022-04-21,2022-04-20,0.012620,0.012509,0
3,dst,103,org,39810,180.06,164.15,RICHARD CROFT d,RICHARD E N BG,0.000022,0.000011,2022-02-11,2022-02-10,0.004622,0.004710,0
4,dst,142,org,142,83503.99,83709.76,AIKATERINI C CHQ,AIKATERINI C C,0.000011,0.000011,2022-05-07,2022-05-03,0.015322,0.013921,0


In [7]:
linker.estimate_u_using_random_sampling(target_rows=1e6)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - amount (no m values are trained).
    - memo (no m values are trained).
    - transaction_date (no m values are trained).


In [8]:
linker.match_weights_chart()

In [9]:
results = linker.predict(threshold_match_probability=0.9)


 -- WARNING --
You have called predict(), but there are some parameter estimates which have neither been estimated or specified in your settings dictionary.  To produce predictions the following untrained trained parameters will use default values.
Comparison: 'amount':
    m values not fully trained
Comparison: 'memo':
    m values not fully trained
Comparison: 'transaction_date':
    m values not fully trained


In [10]:

results.as_pandas_dataframe(limit=10)


,match_weight,match_probability,source_dataset_l,unique_id_l,source_dataset_r,unique_id_r,amount_l,amount_r,gamma_amount,memo_l,memo_r,gamma_memo,transaction_date_l,transaction_date_r,gamma_transaction_date,match_key
0,4.493647,0.957497,dst,696,org,696,568.15,568.15,1,DIEGO D S Y GUZ,DIEGO D S Y GUZ,4,2022-03-27,2022-03-21,0,0
1,4.493647,0.957497,dst,735,org,735,163.15,163.15,1,EMMANUEL PHILIB,EMMANUEL PHILIB,4,2022-05-03,2022-05-01,0,0
2,4.493647,0.957497,dst,882,org,882,423.91,423.91,1,M Q O S donatio,M Q O S donatio,4,2022-04-29,2022-04-27,0,0
3,4.493647,0.957497,dst,273,org,273,36.29,36.29,1,EUCHARIUS RÖSSL,EUCHARIUS RÖSSL,4,2022-04-24,2022-04-23,0,0
4,5.493647,0.978287,dst,576,org,576,42.30,42.30,1,GRUFFYDD D BGC,GRUFFYDD D BGC,4,2022-04-23,2022-04-21,0,0
5,4.908684,0.967781,dst,866,org,866,396.93,396.93,1,M Q O S payment,M Q O S payment,4,2022-05-06,2022-05-04,0,0
6,5.493647,0.978287,dst,369,org,369,27.38,27.38,1,W Z money CHQ,W Z money CHQ,4,2022-02-07,2022-02-06,0,0
7,5.493647,0.978287,dst,397,org,397,436.87,436.87,1,I LOSA BGC,I LOSA BGC,4,2022-03-20,2022-03-08,0,0
8,4.493647,0.957497,dst,414,org,414,190.94,190.94,1,N R,N R,4,2022-02-02,2022-02-01,0,0
9,4.493647,0.957497,dst,439,org,439,111.24,111.24,1,F R money BGC,F R money BGC,4,2022-01-30,2022-01-29,0,0
